## 1. Import Dependencies

In [1]:
# !pip install stable-baselines3[extra]

In [2]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## 2.1 Load Environment

In [3]:
env_name = "CartPole-v0"

In [4]:
env = gym.make(env_name)

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:42.0
Episode:2 Score:12.0
Episode:3 Score:11.0
Episode:4 Score:19.0
Episode:5 Score:21.0


In [6]:
env.close()

In [7]:
episodes = 5
for episodes in range(1, episodes+1):
    print(episodes)

1
2
3
4
5


## 2.2 Understanding The Environment

In [ ]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

In [ ]:
env.action_space

In [ ]:
env.reset()

In [ ]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

In [ ]:
env.observation_space

In [ ]:
env.step(1)

## 3. Train an RL Model

In [ ]:
env_name = "CartPole-v0"

In [ ]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

In [ ]:
model.learn(total_timesteps=20000)

In [ ]:
# PPO??

## 4. Save and Reload Model

In [ ]:
import os

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
# PPO_path

In [ ]:
model.save(PPO_path)

In [ ]:
# del model

In [ ]:
# model.learn(total_timesteps=1000)

In [ ]:
model = PPO.load(PPO_path, env=env)

In [ ]:
# model = PPO.load('PPO_model', env=env)

## 5. Evaluation

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

## 6. Test Model

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

In [ ]:
env.close()

## 7. Viewing Logs in Tensorboard

In [ ]:
log_path = os.path.join('Training', 'Logs')
training_log_path = os.path.join(log_path, 'PPO_1')

In [ ]:
training_log_path

In [ ]:
!tensorboard --logdir={training_log_path}

## 8. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

## 9. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

## 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()